# ASUM-DM Methodology Analysis - Caso 1: Clustering

## 1. Etapas de Entendimiento

### 1.1 Comprensión del Negocio
**Objetivo:** Definir el problema, los objetivos de negocio y los requisitos de la solución.

**Problema:** Una empresa de retail busca implementar estrategias de marketing diferenciadas segmentando a sus clientes según su comportamiento.

**Objetivo de Negocio:** Confirmar o rechazar la hipótesis del equipo de mercadeo de que los clientes se pueden dividir en **3 grupos**.

### 1.2 Enfoque Analítico
**Tipo de Problema:** Aprendizaje no supervisado (Clustering).

**Métodos de Construcción:** Se utilizarán y compararán cuatro algoritmos de agrupamiento:
1. **K-Means:** Método de partición basado en centroides
2. **Método Jerárquico Aglomerativo (MJA):** Clustering jerárquico con dendrogramas
3. **DBSCAN:** Método basado en densidad
4. **Gaussian Mixture Models (GMM):** Método basado en distribución probabilística

## 2. Etapas de Preparación

### 2.1 Recopilación y Carga de Datos

In [ ]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append(os.path.abspath('..'))

from utils.entrega1.data_loader import load_data

df = load_data('../data/datos_caso_1.csv')
df.info()

### 2.2 Comprensión de Datos - Análisis Exploratorio (EDA)

#### Estadísticas Básicas

In [ ]:
from utils.entrega1.eda import get_basic_stats

display(get_basic_stats(df).T)

#### Valores Faltantes

In [ ]:
from utils.entrega1.eda import check_missing_values_viz

check_missing_values_viz(df)

#### Distribuciones de Variables Numéricas

In [ ]:
from utils.entrega1.eda import plot_distributions_numerical

numerical_cols = df.select_dtypes(include=['number']).columns.tolist()
plot_distributions_numerical(df, numerical_cols)

#### Boxplots para Detección de Outliers

In [ ]:
from utils.entrega1.eda import plot_boxplots

plot_boxplots(df, numerical_cols)

#### Variables Categóricas

In [ ]:
from utils.entrega1.eda import plot_pie_categorical

plot_pie_categorical(df, ['Education', 'Marital_Status'])

#### Matriz de Correlación

In [ ]:
from utils.entrega1.eda import plot_correlation_matrix

plot_correlation_matrix(df)

### 2.3 Preprocesamiento de Datos

Aplicamos el pipeline de preprocesamiento que incluye:
- Manejo de valores faltantes
- Eliminación y transformación de valores atípicos
- Ingeniería de características (Age, Days_Being_Customer, Total_Mnt, etc.)
- Codificación de variables categóricas (One-Hot Encoding)
- Escalado de características numéricas (StandardScaler)

In [ ]:
from utils.entrega1.preprocessing import preprocess_pipeline

(
    df_scaled,
    scaler,
    encoder,
    all_feature_cols,
    num_feature_cols,
    ohe_feature_cols,
    categorical_cols,
) = preprocess_pipeline(df)

print(f'Dataset después del preprocesamiento: {df_scaled.shape}')
print(f'Características numéricas: {len(num_feature_cols)}')
print(f'Características OHE: {len(ohe_feature_cols)}')
print(f'Total de características: {len(all_feature_cols)}')

## 3. Modelado - K-Means

### 3.1 Método del Codo y Silhouette Score

Determinamos el número óptimo de clusters analizando la inercia y el Silhouette Score.
La línea roja punteada indica la hipótesis de negocio (k=3).

In [ ]:
from sklearn.cluster import KMeans
from utils.entrega1.modeling import evaluate_clusters_kmeans

evaluate_clusters_kmeans(
    df_scaled,
    range_n_clusters=range(2, 11),
    include_silhouette=True,
    ref_cluster=3,
)

### 3.2 Entrenamiento del Modelo K-Means Final

In [ ]:
n_clusters_kmeans = 3
kmeans_final = KMeans(n_clusters=n_clusters_kmeans, random_state=42, n_init=10)
kmeans_labels = kmeans_final.fit_predict(df_scaled)

print('K-Means - Distribución de clusters:')
print(pd.Series(kmeans_labels).value_counts().sort_index())

### 3.3 Interpretación de Clusters K-Means

Analizamos los centroides de cada cluster en su escala original mediante transformación inversa.

In [ ]:
# Centros escalados
centers_scaled = pd.DataFrame(kmeans_final.cluster_centers_, columns=all_feature_cols)

# Transformación inversa - numéricas
centers_num_inverse = scaler.inverse_transform(centers_scaled[num_feature_cols])
centers_num_df = pd.DataFrame(centers_num_inverse, columns=num_feature_cols)

# Transformación inversa - categóricas
centers_cat_inverse = encoder.inverse_transform(centers_scaled[ohe_feature_cols])
centers_cat_df = pd.DataFrame(centers_cat_inverse, columns=categorical_cols)

# Combinar
centers_original = pd.concat([centers_num_df, centers_cat_df], axis=1)

print('=== Centroides de K-Means (Escala Original) ===')
display(centers_original.T)

## 4. Modelado - Método Jerárquico Aglomerativo (MJA)

### 4.1 Dendrograma

Visualizamos la jerarquía de clusters para determinar el número óptimo de grupos.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from utils.entrega1.modeling import plot_dendrogram

# Para el dendrograma necesitamos el árbol completo
hierarchical_full = AgglomerativeClustering(
    distance_threshold=0,
    n_clusters=None,
    linkage='ward',
)
hierarchical_full.fit(df_scaled)

plt.figure(figsize=(14, 8))
plt.title('Dendrograma - Método Jerárquico Aglomerativo')
plot_dendrogram(hierarchical_full, truncate_mode='level', p=5)
plt.xlabel('Índice de la muestra (o tamaño del cluster)')
plt.show()

### 4.2 Entrenamiento del Modelo Jerárquico

In [ ]:
n_clusters_hierarchical = 3
hierarchical = AgglomerativeClustering(
    n_clusters=n_clusters_hierarchical,
    linkage='ward',
)
hierarchical_labels = hierarchical.fit_predict(df_scaled)

print('MJA - Distribución de clusters:')
print(pd.Series(hierarchical_labels).value_counts().sort_index())

## 5. Modelado - DBSCAN

### 5.1 Estimación de Epsilon (k-NN Distance)

Utilizamos el método del k-NN distance para estimar el valor de epsilon.

In [ ]:
from sklearn.cluster import DBSCAN
from utils.entrega1.modeling import plot_knn_distance, optimize_dbscan_grid

plot_knn_distance(df_scaled, k=5)

### 5.2 Optimización de Hiperparámetros (Grid Search)

In [ ]:
import numpy as np

dbscan_results_df = optimize_dbscan_grid(
    df_scaled,
    eps_values=np.arange(1.0, 5.0, 0.5),
    min_samples_values=[3, 5, 7, 10],
)

# Seleccionar mejores parámetros
best_row = dbscan_results_df.loc[dbscan_results_df['Score'].idxmax()]
best_eps = best_row['Epsilon']
best_min_samples = int(best_row['Vecindad'])
print(f'Mejores parámetros: eps={best_eps}, min_samples={best_min_samples}, score={best_row["Score"]:.4f}')

### 5.3 Entrenamiento del Modelo DBSCAN

In [ ]:
dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
dbscan_labels = dbscan.fit_predict(df_scaled)

print('DBSCAN - Distribución de clusters:')
print(pd.Series(dbscan_labels).value_counts().sort_index())
print(f'\nPuntos de ruido (label=-1): {(dbscan_labels == -1).sum()}')

## 6. Modelado - Gaussian Mixture Models (GMM)

### 6.1 Selección del Número de Componentes (BIC)

Utilizamos el Criterio de Información Bayesiano (BIC) para determinar el número óptimo de componentes.

In [ ]:
from sklearn.mixture import GaussianMixture
from utils.entrega1.modeling import evaluate_gmm_bic

evaluate_gmm_bic(
    df_scaled,
    n_components_range=range(2, 11),
)

### 6.2 Entrenamiento del Modelo GMM

In [ ]:
n_components_gmm = 3
gmm = GaussianMixture(
    n_components=n_components_gmm,
    covariance_type='full',
    random_state=42,
)
gmm_labels = gmm.fit_predict(df_scaled)

print('GMM - Distribución de clusters:')
print(pd.Series(gmm_labels).value_counts().sort_index())

## 7. Comparación de Modelos

### 7.1 Silhouette Score

Comparamos todos los modelos utilizando el Silhouette Score.

In [ ]:
from utils.entrega1.modeling import compare_all_models_silhouette

models_dict = {
    'K-Means': kmeans_labels,
    'Jerárquico': hierarchical_labels,
    'DBSCAN': dbscan_labels,
    'GMM': gmm_labels,
}

scores = compare_all_models_silhouette(df_scaled, models_dict)

### 7.2 Visualización con PCA

Reducimos la dimensionalidad a 2D para visualizar los clusters de cada modelo.

In [ ]:
from utils.entrega1.modeling import visualize_clusters_pca

for name, labels in models_dict.items():
    visualize_clusters_pca(df_scaled, labels, title=f'{name} Clusters (PCA)')

## 8. Conclusiones

### 8.1 Resumen de Resultados

Basándonos en el análisis realizado:

1. **K-Means** y **Método Jerárquico** confirman la hipótesis de negocio de 3 clusters
2. **DBSCAN** identifica clusters basados en densidad y puede detectar outliers
3. **GMM** proporciona asignación probabilística a los clusters

### 8.2 Recomendaciones

- El modelo **K-Means** es recomendado por su simplicidad e interpretabilidad
- Los 3 clusters identificados permiten estrategias de marketing diferenciadas
- Se recomienda validar los resultados con el equipo de negocio